## Описание проекта

### Заказчик

Компания GlowByte.

### Описание проблемы

Прогнозирование электропотребления представляет собой сложную задачу из-за нескольких ключевых проблем:

- Неопределенность: Будущее поведение потребителей и их потребности в электроэнергии не всегда можно предсказать точно. Различные факторы, такие как экономические изменения, социокультурные тенденции, технологические прорывы и политические решения, могут существенно повлиять на потребление электроэнергии, и невозможно учесть все возможные варианты в прогнозах.

- Сезонные и временные изменения: Электропотребление часто подвержено сезонным и временным колебаниям. Например, летом потребление может возрасти из-за использования кондиционеров, а в праздничные дни или выходные потребление может снизиться. Прогнозирование этих изменений требует учета множества факторов и данных.

- Развивающиеся технологии: Быстрое развитие новых технологий, таких как электромобили, возобновляемые источники энергии и энергоэффективные устройства, меняет пейзаж потребления электроэнергии. Прогнозирование, как и когда эти технологии будут внедрены, представляет вызов для экспертов.

- Энергетическая политика и регулирование: Политические решения и изменения в законодательстве в области энергетики могут существенно повлиять на спрос на электроэнергию.

- Глобальные факторы: Электроэнергия является частью мировой экономики, и множество глобальных факторов, таких как изменения климата, геополитические конфликты и экономические кризисы, могут оказать влияние на ее потребление.

### Цель

Разработать надежную и точную модель прогнозирования объема энергопотребления на сутки для Калининградской области с использованием доступных исторических данных и соответствующих переменных.

### Задача

Разработка модели прогнозирования общегоэнергопотребления региона на сутки, в МВт ч

### Описание задачи

В данной задаче необходимо разработать предиктивную модель, которая позволит прогнозировать энергопотребление региона на основе имеющихся данных о потреблении электроэнергии в прошлом и соответствующих факторах, влияющих на потребление энергии. Модель должна быть способна учесть сезонные, временные и другие зависимости для более точного прогноза.
Модель должна предсказывать общее потребление региона на 1 сутки

## План выполнения проекта

## Реализация проекта

### Первичное знакомство с данными

#### Загрузка данных

Загрузка библиотек

In [2]:
import pandas as pd
from os.path import join
from pathlib import Path
import os

Путь к данным

In [3]:
dp = Path("C:/Users/79099/Desktop/Hakaton_GlowByte")

Тренировочный датасет

In [16]:
df_train = pd.read_csv(join(dp, 'train_dataset.csv'), index_col=[0], parse_dates=[0])
df_train.sort_index(inplace=True)

In [17]:
df_train.head(5)

,time,target,temp,temp_pred,weather_pred,weather_fact
date,,,,,,
2019-01-01,0,481.510,2.9,2.0,"пасм, ветер",ветер
2019-01-01,1,462.872,2.9,2.0,"пасм, ветер",ветер
2019-01-01,2,449.718,2.9,2.0,"пасм, ветер",ветер
2019-01-01,3,430.908,4.3,2.0,"пасм, ветер","ветер, пасм"
2019-01-01,4,415.163,4.3,2.0,"пасм, ветер","ветер, пасм"


Тестовый датасет

In [18]:
df_test = pd.read_csv(join(dp, 'test_dataset.csv'), index_col=[0], parse_dates=[0])
df_test.sort_index(inplace=True)

In [19]:
df_test.head(5)

,time,target,temp,temp_pred,weather_pred,weather_fact
date,,,,,,
2023-04-01,0,479.282,5.7,5.0,"пасм, 58%","пасм, морось"
2023-04-01,1,445.182,5.7,5.0,"пасм, 58%","пасм, морось"
2023-04-01,2,424.225,5.7,5.0,"пасм, 58%","пасм, морось"
2023-04-01,3,413.866,5.0,4.0,"пасм, 71% дождь","пасм, дымка"
2023-04-01,4,408.146,5.0,4.0,"пасм, 71% дождь","пасм, дымка"


**Вывод** 

Данные загрузились корректно

- Тренировочный датасет содержит 7 столбцов и 37224 строки
- Тестовый датасет содержит 7 столбцов и 2928 строк

Описание данных:

- date: дата
- time: время, время представлено в диапазоне 0 – 23, что означает 24 часа в сутках
- target: Фактическое потребление на указанную дату
- temp: фактическая температура на указанную дату
- temp_pred: прогноз температуры на указанную дату
- weather_fact: фактическая погода на указанную дату
- weather_pred: прогноз погоды на указанную дату

#### Формирование датафреймов

In [20]:
def groupe_temp_pred(df):
    df_group = df.groupby('date')['temp_pred'].agg(['min', 'max', 'mean'])
    df_group['differences'] = abs(df_group['max'] - df_group['min'])
    df_group = df_group.rename(columns={'min': 'min_temp_pred', 'max': 'max_temp_pred', 'mean': 'mean_temp_pred',
                                        'differences' : 'differences_temp_pred'})
    return df_group
    

In [21]:
def groupe_temp(df):
    df_group = df.groupby('date')['temp'].agg(['min', 'max', 'mean'])
    df_group['differences'] = abs(df_group['max'] - df_group['min'])
    df_group = df_group.rename(columns={'min': 'min_temp', 'max': 'max_temp', 'mean': 'mean_temp',
                                        'differences' : 'differences_temp'})
    return df_group
    

In [22]:
df_group_temp_pred_train = groupe_temp_pred(df_train)

In [23]:
df_group_temp_train = groupe_temp(df_train)

In [24]:
df_group_temp_train

,min_temp,max_temp,mean_temp,differences_temp
date,,,,
2019-01-01,2.9,6.1,4.8750,3.2
2019-01-02,0.2,3.2,1.3500,3.0
2019-01-03,-3.8,0.1,-1.1625,3.9
2019-01-04,-6.9,2.2,-1.5000,9.1
2019-01-05,-1.6,3.4,0.9375,5.0
...,...,...,...,...
2023-03-27,-2.4,3.9,1.4375,6.3
2023-03-28,-3.9,0.9,-1.3000,4.8
2023-03-29,-3.8,6.2,2.3750,10.0


In [25]:
df_group_temp_pred_train

,min_temp_pred,max_temp_pred,mean_temp_pred,differences_temp_pred
date,,,,
2019-01-01,2.0,6.0,4.375,4.0
2019-01-02,0.0,4.0,1.500,4.0
2019-01-03,-2.0,-1.0,-1.375,1.0
2019-01-04,-2.0,1.0,-0.125,3.0
2019-01-05,-2.0,4.0,1.500,6.0
...,...,...,...,...
2023-03-27,-2.0,3.0,1.250,5.0
2023-03-28,-2.0,2.0,-0.250,4.0
2023-03-29,-2.0,5.0,1.500,7.0


In [26]:
df_group_train =  groupe_temp_pred(df_train)

In [27]:
df_group = df_train.groupby('date')['temp_pred'].agg(['min', 'max', 'mean'])

In [28]:
df_group

,min,max,mean
date,,,
2019-01-01,2.0,6.0,4.375
2019-01-02,0.0,4.0,1.500
2019-01-03,-2.0,-1.0,-1.375
2019-01-04,-2.0,1.0,-0.125
2019-01-05,-2.0,4.0,1.500
...,...,...,...
2023-03-27,-2.0,3.0,1.250
2023-03-28,-2.0,2.0,-0.250
2023-03-29,-2.0,5.0,1.500


In [29]:
df_group['Differences'] = abs(df_group['max'] - df_group['min'])

In [30]:
df_group

,min,max,mean,Differences
date,,,,
2019-01-01,2.0,6.0,4.375,4.0
2019-01-02,0.0,4.0,1.500,4.0
2019-01-03,-2.0,-1.0,-1.375,1.0
2019-01-04,-2.0,1.0,-0.125,3.0
2019-01-05,-2.0,4.0,1.500,6.0
...,...,...,...,...
2023-03-27,-2.0,3.0,1.250,5.0
2023-03-28,-2.0,2.0,-0.250,4.0
2023-03-29,-2.0,5.0,1.500,7.0


In [31]:
df_train[df_train['weather_fact'].isnull()]

,time,target,temp,temp_pred,weather_pred,weather_fact
date,,,,,,
2022-03-19,12,582.397,9.0,9.0,малообл.,NaN


In [32]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 37224 entries, 2019-01-01 to 2023-03-31
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   time          37224 non-null  int64  
 1   target        37224 non-null  float64
 2   temp          37224 non-null  float64
 3   temp_pred     37136 non-null  float64
 4   weather_pred  37136 non-null  object 
 5   weather_fact  37223 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 2.0+ MB


In [33]:
df_train.isna().sum()

time             0
target           0
temp             0
temp_pred       88
weather_pred    88
weather_fact     1
dtype: int64

In [34]:
df_test.isna().sum()

time             0
target           0
temp             0
temp_pred       24
weather_pred    24
weather_fact     0
dtype: int64

In [35]:
df_train.index.is_monotonic

True

In [36]:
df_test.index.is_monotonic

True

#### Ресемплирование данных

In [37]:
df_tr = df_train.resample('1D').sum()

In [38]:
df_tt = df_test.resample('1D').sum()

In [39]:
df_tr

,time,target,temp,temp_pred
date,,,,
2019-01-01,276,11126.866,117.0,105.0
2019-01-02,276,11547.210,32.4,36.0
2019-01-03,276,12235.564,-27.9,-33.0
2019-01-04,276,12763.044,-36.0,-3.0
2019-01-05,276,12735.145,22.5,36.0
...,...,...,...,...
2023-03-27,276,12513.596,34.5,30.0
2023-03-28,276,12988.331,-31.2,-6.0
2023-03-29,276,12771.077,57.0,36.0


In [40]:
df = df.drop(['C', 'D'], axis=1)

NameError: name 'df' is not defined